<a href="https://colab.research.google.com/github/asthabaj/Complete-Python-3-Bootcamp/blob/master/imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from google.colab import files
import matplotlib.pyplot as plt


In [ ]:
transform = transforms.Compose ([ #scale 0-255 into -1 to 1 and tensors
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])


In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)
train_loder = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

100%|██████████| 170M/170M [00:04<00:00, 35.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
image, label = train_data[0] #to see structure
print(image.shape, label)

torch.Size([3, 32, 32]) 6


# 3 channels rgb, 32 x 32 pixel

In [ ]:
class_name = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck'] #0-9 classes

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self): #constructor
    super(). __init__()

    self.conv1 = nn.Conv2d(3, 12, 5) #12,28,28
    self.pool = nn.MaxPool2d(2, 2) #12,14,14
    self.conv2 = nn.Conv2d(12, 24, 5) #24,10,10 -> 24,5,5 -> flatten(24*5*5)
    self.fc1 = nn.Linear(24*5*5,120) #agai ko 3 ta shape,120 neurons o/p
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10) #change values bichk, but in the end 10 huna parcha for 10 classes

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x




3 ip channel/12 op channel/kernel size to extract features
ip size - kernel size = 32-5=27/stride(1)+1
new shape after the convolutional layer is 12 channels 28x28 pixel
max pool le 2x2 pixel area lai 1 pixel extracting most imp feature, divides by 2 to picture
sec conv layer, 12 ip channel, 24 op with 5x5 kernel
14 pixel size -5 /stride +1 =10





feed it to first layer, activation function relu, if ip is 0 if less than 0 and input if >0,
activation layer breaks linearily


In [ ]:
#or training the network
net = NeuralNetwork()
loss_function = nn.CrossEntropyLoss()
optimizer =  optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #stochastic gradient descent


In [ ]:
for epoch in range(30):
  print(f'Training epoch {epoch}..')
  running_loss = 0.0

  for i, data in enumerate(train_loder):
    inputs, labels = data
    optimizer.zero_grad() #set all gradients to none
    outputs = net(inputs)

    loss = loss_function(outputs, labels)
    loss.backward() #backward prop
    optimizer.step()

    running_loss += loss.item()
  print(f'Loss: {running_loss/len(train_loder):.4f}')

Training epoch 0..
Loss: 2.1794
Training epoch 1..
Loss: 1.7779
Training epoch 2..
Loss: 1.5363
Training epoch 3..
Loss: 1.4081
Training epoch 4..
Loss: 1.3127
Training epoch 5..
Loss: 1.2347
Training epoch 6..
Loss: 1.1649
Training epoch 7..
Loss: 1.1085
Training epoch 8..
Loss: 1.0591
Training epoch 9..
Loss: 1.0092
Training epoch 10..
Loss: 0.9683
Training epoch 11..
Loss: 0.9300
Training epoch 12..
Loss: 0.8934
Training epoch 13..
Loss: 0.8571
Training epoch 14..
Loss: 0.8269
Training epoch 15..
Loss: 0.7950
Training epoch 16..
Loss: 0.7638
Training epoch 17..
Loss: 0.7349
Training epoch 18..
Loss: 0.7099
Training epoch 19..
Loss: 0.6813
Training epoch 20..
Loss: 0.6580
Training epoch 21..
Loss: 0.6311
Training epoch 22..
Loss: 0.6084
Training epoch 23..
Loss: 0.5894
Training epoch 24..
Loss: 0.5624
Training epoch 25..
Loss: 0.5459
Training epoch 26..
Loss: 0.5226
Training epoch 27..
Loss: 0.4992
Training epoch 28..
Loss: 0.4841
Training epoch 29..
Loss: 0.4641


In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')
print("Model weights saved as 'trained_net.pth'") #export parameters

Model weights saved as 'trained_net.pth'


In [ ]:
files.download('trained_net.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#net = NeuralNetwork() #to load in another script, exporting weights and biases
#net.load_state_dict(torch.load('trained_net.pth'))

<ipython-input-14-ce7766172526>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [ ]:
# upload the model weights file to Colab
uploaded = files.upload()
model_file = 'trained_net.pth'  # Ensure you upload this file"
net.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
net.eval()

Saving trained_net.pth to trained_net (1).pth


<ipython-input-22-f9fa60b075df>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))


NeuralNetwork(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
#evaluate on test data
correct = 0
total = 0

net.eval() #during evaluation bcus model works different
with torch.no_grad(): #not do any gradient conputations bcus not training
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs,1) #highest 1 instance with activation
    total += labels.size(0) #32 instances we classified
    correct += (predicted == labels).sum().item()

print(f'Accuracy: {100*correct/total}%')

Accuracy: 68.59%


In [ ]:
#testing
new_transform = transforms.Compose([
  transforms.Resize((32,32)), #resize to 32 pixels
  transforms.ToTensor(),
  transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0) #add batch dimension to be compatible
  return image

print("Please upload the images you want to classify:")
uploaded_images = files.upload()
image_paths = list(uploaded_images.keys())

net.eval()
with torch.no_grad():
    for path in image_paths:
        image = load_image(path)
        output = net(image)
        _, predicted = torch.max(output, 1)
    print("the image is of",class_name[predicted])



Please upload the images you want to classify:


Saving astha bajracharya1.jpg to astha bajracharya1.jpg
the image is of cat
